In [1]:
import pandas as pd
import folium
import numpy as np
import cPickle as pickle
import pdb
from selenium import webdriver

In [2]:
# Read data
data_fname = r'temperature_mex_gov.p'
map_data_fname = r'MEX_adm1_tunned.csv'
mex_states_geo = r'mexitest700.geojson'


temperature = pd.read_pickle(data_fname)
mex_admin_div = pd.read_csv(map_data_fname)

In [3]:
# Make data and geo data files compatible
estados_nombres = ['Aguascalientes',
                   'Baja California',
                   'Baja California Sur',
                   'Campeche',
                   'Chiapas',
                   'Chihuahua',
                   'Coahuila',
                   'Colima',
                   'DF',
                   'Durango',
                   'Guanajuato',
                   'Guerrero',
                   'Hidalgo',
                   'Jalisco',
                   'Michoacán',
                   'Morelos',
                   'México',
                   'Nayarit',
                   'Nuevo León',
                   'Oaxaca',
                   'Puebla',
                   'Querétaro',
                   'Quintana Roo',
                   'San Luis Potosí',
                   'Sinaloa',
                   'Sonora',
                   'Tabasco',
                   'Tamaulipas',
                   'Tlaxcala',
                   'Veracruz',
                   'Yucatán',
                   'Zacatecas']


In [4]:
def trans_dict(A, B):
    """
    Creates translation dictionary 
    from A to B
    """
    td = {}
    for i, k in enumerate(A):
        td[k] = B[i]
    return td

def encode_names(trans_dict, names):
    """
    """
    if hasattr(names, "__iter__"):
        enc_names = []
        for name in names:
            enc_names.append(trans_dict[name])
    else:
        enc_names = trans_dict[names]    
    return enc_names



In [5]:
temperature.ENTIDAD[:32].values.astype('str')

array(['AGUASCALIENTES', 'BAJA CALIFORNIA', 'BAJA CALIFORNIA SUR',
       'CAMPECHE', 'COAHUILA  DE ZARAGOZA', 'COLIMA', 'CHIAPAS',
       'CHIHUAHUA', 'DISTRITO FEDERAL', 'DURANGO', 'GUANAJUATO',
       'GUERRERO', 'HIDALGO', 'JALISCO', 'M\xc9XICO',
       'MICHOAC\xc1N DE OCAMPO', 'MORELOS', 'NAYARIT', 'NUEVO LE\xd3N',
       'OAXACA', 'PUEBLA', 'QUER\xc9TARO', 'QUINTANA ROO',
       'SAN LUIS POTOS\xcd', 'SINALOA', 'SONORA', 'TABASCO', 'TAMAULIPAS',
       'TLAXCALA', 'VERACRUZ DE IGNACIO DE LA LLAVE', 'YUCAT\xc1N',
       'ZACATECAS'], 
      dtype='|S31')

In [6]:
#short_names = trans_dict(estados_nombres,mex_admin_div.HASC_1)
short_names = trans_dict(temperature.ENTIDAD[:32].astype('str'), mex_admin_div.HASC_1)

def encode_mx_names(name):
    return encode_names(short_names, name)

In [7]:
temperature['HASC_1'] = temperature.ENTIDAD.apply(encode_mx_names)
temperature.rename(columns={'TEMPERATURA':'maxtmp'}, inplace=True)
temperature.rename(columns={'AÑO':'YEAR'}, inplace=True)

In [8]:
statell = ((21.8853, -102.2915),
           (30.8406, -115.2838),
           (26.0444, -111.6661),
           (18.9312, -90.2618 ),
           (27.0587, -101.7068),
           (19.1223, -104.0072),
           (16.7569, -93.1292),
           (28.6330, -106.0691),
           (19.4326, -99.1332),                      
           (24.2753, -104.8801),#Google is wrong
           (21.0190, -101.2574),
           (17.4392, -99.5451),           
           (20.0911, -98.7624),           
           (20.6595, -103.3494),           
           (19.4969, -99.7233),           
           (19.5665, -101.7068),           
           (18.6813, -99.1013),           
           (21.7514, -104.8455),           
           (25.5922, -99.9962),           
           (17.0542, -96.7132),           
           (19.0413, -98.2062),           
           (20.5888, -100.3899),           
           (19.1817, -88.4791),           
           (22.1566, -100.9855),           
           (25.1721, -107.4795),           
           (29.2972, -110.3309),           
           (17.8409, -92.6189),           
           (24.2669, -98.8363),           
           (19.3182, -98.2375),
           (19.2602, -96.5783),
           (20.7099, -89.0943),
           (22.7709, -102.5832)           
           )

In [9]:
lat, lon = zip(*statell)

In [10]:
lats = {}
lons = {}
for i, k in enumerate(temperature.HASC_1[:32]):
    lats[k] = lat[i]
    lons[k] = lon[i]    

In [11]:
lons

{'MX.AG': -102.2915,
 'MX.BN': -111.6661,
 'MX.BS': -115.2838,
 'MX.CA': -93.1292,
 'MX.CH': -104.0072,
 'MX.CL': -106.0691,
 'MX.CM': -90.2618,
 'MX.CP': -101.7068,
 'MX.DF': -99.1332,
 'MX.DU': -104.8801,
 'MX.GJ': -101.2574,
 'MX.GR': -99.5451,
 'MX.HI': -98.7624,
 'MX.JA': -103.3494,
 'MX.MC': -101.7068,
 'MX.MR': -99.1013,
 'MX.MX': -99.7233,
 'MX.NA': -104.8455,
 'MX.NL': -99.9962,
 'MX.OA': -96.7132,
 'MX.PU': -98.2062,
 'MX.QE': -100.3899,
 'MX.QR': -88.4791,
 'MX.SI': -107.4795,
 'MX.SL': -100.9855,
 'MX.SO': -110.3309,
 'MX.TB': -92.6189,
 'MX.TL': -98.2375,
 'MX.TM': -98.8363,
 'MX.VE': -96.5783,
 'MX.YU': -89.0943,
 'MX.ZA': -102.5832}

In [12]:
def mex_temp(year, month, temp_data=temperature):
    """
    """    
    if month in temp_data['MES_int'].values:    
        tmp = temp_data[(temperature['YEAR'] == year)  & (temperature['MES_int'] == month)]
    elif month in temp_data['MES'].values:    
        tmp = temp_data[(temperature['YEAR'] == year)  & (temperature['MES'] == month)]
            
    return tmp

    

In [13]:
def add_markers(sel_map,
                data,
                lats,
                lons,
                debug=False,
                **kwargs):
    """
    """
    for k in lats:
        tmp_val = data['maxtmp'][data['HASC_1'] == str(k)].values
        state = data['ENTIDAD'][data['HASC_1'] == k].values
        marker_legend = unicode(str(state[0]) + ': '+ str(tmp_val[0]),
                                errors='ignore')
        if debug:
            print marker_legend
        mk = folium.Marker(location=[lats[k], lons[k]],
                           popup=marker_legend)
        mk.add_to(sel_map)

In [20]:
%qtconsole

In [14]:
def create_map(year, 
               month,
               markers=True,
               save=False,
               debug=False,
               **kwargs):
    """
    """
    opt = {}
    opt['name'] = 'noname'
    opt.update(kwargs)
    
    sel_data = mex_temp(year, month)
    map_mx = folium.Map(location=[23.53, -102.55], zoom_start=4.5)

    map_mx.choropleth(geo_path=mex_states_geo,
                      data=sel_data,
                      columns=['HASC_1', 'maxtmp'],
                      key_on='feature.properties.HASC_1',
                      fill_color='OrRd',
                      fill_opacity=0.7,
                      line_opacity=0.2,
                      legend_name='Temperature')
    if markers:
        add_markers(map_mx,
                    sel_data,
                    lats,
                    lons,
                    debug,
                    **kwargs)
    if save is True:
        map_mx.save(opt['name']+'.html')
        if screen is True:
            browser = webdriver.Firefox()
            browser.get(opt['name']+'.html')
            #Give the map tiles some time to load
            time.sleep(5)
            browser.save_screenshot('map.png')
            browser.quit()
    return map_mx

In [19]:
screen=True
map_mx_new = create_map(year=1986,
                        month=9,
                        markers=False,
                        save=True,
                        debug=False,
                        screen=True)

/Applications/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:23: FutureWarning: 'threshold_scale' default behavior has changed. Now you get a linear scale between the 'min' and the 'max' of your data. To get former behavior, use folium.utilities.split_six.


WebDriverException: Message: 'Can\'t load the profile. Profile Dir: /var/folders/d0/h1123zw10cl569yx765cpxp00000gn/T/tmpwIw21t Firefox output: 1468432793766\taddons.manager\tDEBUG\tLoaded provider scope for resource://gre/modules/addons/XPIProvider.jsm: ["XPIProvider"]\n1468432793768\taddons.manager\tDEBUG\tLoaded provider scope for resource://gre/modules/LightweightThemeManager.jsm: ["LightweightThemeManager"]\n1468432793770\taddons.manager\tDEBUG\tLoaded provider scope for resource://gre/modules/addons/GMPProvider.jsm\n1468432793772\taddons.manager\tDEBUG\tLoaded provider scope for resource://gre/modules/addons/PluginProvider.jsm\n1468432793772\taddons.manager\tDEBUG\tStarting provider: XPIProvider\n1468432793773\taddons.xpi\tDEBUG\tstartup\n1468432793773\taddons.xpi\tINFO\tMapping fxdriver@googlecode.com to /var/folders/d0/h1123zw10cl569yx765cpxp00000gn/T/tmpwIw21t/extensions/fxdriver@googlecode.com\n1468432793774\taddons.xpi\tINFO\tSystemAddonInstallLocation directory is missing\n1

In [16]:
map_mx_new

NameError: name 'map_mx_new' is not defined

In [ ]:
map_mx_new2 = create_map(year=1986,
                        month=5,
                        markers=True,
                        save=False,
                        debug=False)

In [ ]:
map_mx_new2

In [ ]:
# Gen maps for all years
mx_maps = {}
for year in temperature['YEAR'].unique():
    mx_maps[year] = {}    
    for month in temperature['MES'].unique():
        name = str(year) + '_' + str(month)
        print year
        print month
        mx_maps[year][month] = create_map(year=year,
                                          month=month,
                                          markers=True,
                                          save=True,                                          
                                          name=name)
        

In [ ]:
%%HTML
<iframe width="640" height="480" src="noname.html"></iframe>